In [2]:
!pip install smdebug
!pip install -U pip
!pip install -U setuptools wheel
!pip install matplotlib
!pip3 install kaggle
!pip install torch torchvision
!pip install torch --no-cache-dir

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils 

In [4]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.debugger import Rule, DebuggerHookConfig, ProfilerConfig, FrameworkProfile, Rule, ProfilerRule, rule_configs, TensorBoardOutputConfig
from sagemaker.pytorch import PyTorch
from sagemaker.predictor import Predictor
from sagemaker.tuner import (IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner)
from sagemaker.debugger import CollectionConfig
from sagemaker.pytorch import PyTorchModel
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


#import pytorch
#import torchvision

In [6]:
# create the .kaggle directory and an empty kaggle.json file
!mkdir -p /root/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
# Fill in your user name and key from creating the kaggle account and API token file
import json
kaggle_username = "akpojichekoeyekpegha"
kaggle_key = "f112b616e18d446cebc6bcb2cb35a53d"
# Save API token the kaggle.json file
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

In [9]:
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

the-nature-conservancy-fisheries-monitoring.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -o the-nature-conservancy-fisheries-monitoring.zip

In [ ]:
!unzip train.zip

In [ ]:
!unzip test.stg1.zip

In [5]:
trn_images = []
trn_labels = []

for category in os.listdir("train/"):
    if os.path.isdir("train/%s" % (category)):
        for img in os.listdir("train/%s" % (category)):
            if img.endswith(".jpg"):
                trn_images.append("train/%s/%s" % (category, img))
                trn_labels.append("%s" % (category))

In [6]:
os.listdir("train/")

['LAG', 'OTHER', '.DS_Store', 'SHARK', 'NoF', 'YFT', 'BET', 'DOL', 'ALB']

In [7]:
trn_images[:5]

['train/LAG/img_02694.jpg',
 'train/LAG/img_04953.jpg',
 'train/LAG/img_07176.jpg',
 'train/LAG/img_03774.jpg',
 'train/LAG/img_06223.jpg']

In [7]:
train_df = pd.DataFrame({"images": trn_images, "labels": trn_labels})

train_df.count()

images    3777
labels    3777
dtype: int64

In [9]:
train_df.head(5)

,images,labels
0,train/LAG/img_02694.jpg,LAG
1,train/LAG/img_04953.jpg,LAG
2,train/LAG/img_07176.jpg,LAG
3,train/LAG/img_03774.jpg,LAG
4,train/LAG/img_06223.jpg,LAG


In [6]:
distribution = train_df.groupby("labels")
distribution.count()

,images
labels,
ALB,1719
BET,200
DOL,117
LAG,67
NoF,465
OTHER,299
SHARK,176
YFT,734


In [11]:
This dataframe clearly shows that the test data not alot and it is unbalanced since each classes have widely different numbers of images represented,
preprocessing needs to be done so as to improve its performance during training

SyntaxError: invalid syntax (<ipython-input-11-c393d1a84e0a>, line 1)

In [9]:

sess = sagemaker.Session()
bucket = sess.default_bucket()                                ## TODO: fill in
print("Default Bucket: {}".format(bucket))

my_session = boto3.session.Session()
region = my_session.region_name## TODO: fill in
print("AWS Region: {}".format(region))


role = get_execution_role()## TODO: fill in
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-318398153776
AWS Region: us-east-1
RoleArn: arn:aws:iam::318398153776:role/service-role/AmazonSageMaker-ExecutionRole-20220116T020674


In [ ]:
os.environ["DEFAULT_S3_BUCKET"] = bucket
   
!aws s3 sync ./train s3://${DEFAULT_S3_BUCKET}/train/
!aws s3 sync ./test_stg1 s3://${DEFAULT_S3_BUCKET}/test/

In [20]:
!mkdir ./train_dataset

In [ ]:
hypermarameter tuning

In [10]:
#TODO: Declare your HP ranges, metrics etc.

from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256])
}
objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [12]:
#TODO: Create estimators for your HPs
estimator = PyTorch(
    entry_point="hpo.py",
    base_job_name="image_classification_resneT",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.4.0",
    py_version="py3"
)# TODO: Your estimator here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type
)
# TODO: Your HP tuner here

In [13]:
# TODO: Fit your HP Tuner

s3_data = "s3://{}/".format(bucket)
s3_output_dir = "s3://{}/{}/".format(bucket, "output")
s3_model_dir = "s3://{}/{}/".format(bucket, "model")


os.environ['SM_CHANNEL_TRAIN']=s3_data
os.environ['SM_MODEL_DIR']=s3_model_dir
os.environ['SM_OUTPUT_DATA_DIR']=s3_output_dir


tuner.fit({'train' : s3_data},wait=True) 

#tuner.fit() # TODO: Remember to include your data channels

................................................................................................................................*


UnexpectedStatusException: Error for HyperParameterTuning job pytorch-training-220126-0250: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.